### Exercícios da Aula 04
---
 
Author : Halisson S. Gomides - halisson.gomides@gmail.com

Date : 29/01/2021

---

### Conjunto de dados House Sales in King County, USA
>https://www.kaggle.com/harlfoxem/housesalesprediction


### Respondendo perguntas do CEO:

1. Adicione as seguintes informações ao imóvel:
        - O nome da Rua
        - O número do imóvel
        - O nome do Bairro
        - O nome da Cidade
        - O nome do Estado
        
2. Onde tem essas informações: API chamada GEOPY
3. Fazer o link com 'zipcode' e 'lat' e 'long'

In [1]:
import pandas as pd

In [2]:
dfhouses = pd.read_csv('dados/originais/kc_house_data.csv.zip', 
                       parse_dates=['date'], 
                       infer_datetime_format=True,
                      dtype={'condition': str })

In [3]:
dfhouses.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503


In [4]:
dfhouses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21613 entries, 0 to 21612
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             21613 non-null  int64         
 1   date           21613 non-null  datetime64[ns]
 2   price          21613 non-null  float64       
 3   bedrooms       21613 non-null  int64         
 4   bathrooms      21613 non-null  float64       
 5   sqft_living    21613 non-null  int64         
 6   sqft_lot       21613 non-null  int64         
 7   floors         21613 non-null  float64       
 8   waterfront     21613 non-null  int64         
 9   view           21613 non-null  int64         
 10  condition      21613 non-null  object        
 11  grade          21613 non-null  int64         
 12  sqft_above     21613 non-null  int64         
 13  sqft_basement  21613 non-null  int64         
 14  yr_built       21613 non-null  int64         
 15  yr_renovated   2161

In [5]:
# Verificando se existem indices duplicados

for k,v in dfhouses['id'].value_counts().to_dict().items():
    if v > 1: 
        print(k, v)
        print(dfhouses.loc[dfhouses['id'] == k])

        

795000620 3
              id       date     price  bedrooms  bathrooms  sqft_living  \
17602  795000620 2014-09-24  115000.0         3        1.0         1080   
17603  795000620 2014-12-15  124000.0         3        1.0         1080   
17604  795000620 2015-03-11  157000.0         3        1.0         1080   

       sqft_lot  floors  waterfront  view  ... grade  sqft_above  \
17602      6250     1.0           0     0  ...     5        1080   
17603      6250     1.0           0     0  ...     5        1080   
17604      6250     1.0           0     0  ...     5        1080   

       sqft_basement  yr_built  yr_renovated  zipcode      lat    long  \
17602              0      1950             0    98168  47.5045 -122.33   
17603              0      1950             0    98168  47.5045 -122.33   
17604              0      1950             0    98168  47.5045 -122.33   

       sqft_living15  sqft_lot15  
17602           1070        6250  
17603           1070        6250  
17604       

              id       date      price  bedrooms  bathrooms  sqft_living  \
2564  7856400240 2014-06-27  1620000.0         4        3.0         3900   
2565  7856400240 2015-02-11  1650000.0         4        3.0         3900   

      sqft_lot  floors  waterfront  view  ... grade  sqft_above  \
2564      9750     1.0           0     4  ...    10        2520   
2565      9750     1.0           0     4  ...    10        2520   

      sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
2564           1380      1972             0    98006  47.5605 -122.158   
2565           1380      1972             0    98006  47.5605 -122.158   

      sqft_living15  sqft_lot15  
2564           3410        9450  
2565           3410        9450  

[2 rows x 21 columns]
5101402435 2
               id       date     price  bedrooms  bathrooms  sqft_living  \
11287  5101402435 2014-06-03  312000.0         3       2.25         1540   
11288  5101402435 2015-03-04  539000.0         3       2

               id       date     price  bedrooms  bathrooms  sqft_living  \
12338  5132000140 2014-06-18  175000.0         6        1.0         1370   
12339  5132000140 2015-01-20  415000.0         6        1.0         1370   

       sqft_lot  floors  waterfront  view  ... grade  sqft_above  \
12338      5080     1.5           0     0  ...     6        1120   
12339      5080     1.5           0     0  ...     6        1120   

       sqft_basement  yr_built  yr_renovated  zipcode      lat    long  \
12338            250      1931             0    98106  47.5238 -122.35   
12339            250      1931             0    98106  47.5238 -122.35   

       sqft_living15  sqft_lot15  
12338           1020        5080  
12339           1020        5080  

[2 rows x 21 columns]
8832900780 2
              id       date     price  bedrooms  bathrooms  sqft_living  \
6434  8832900780 2014-10-13  480000.0         5        2.0         1760   
6435  8832900780 2015-04-08  647500.0         5     

             id       date     price  bedrooms  bathrooms  sqft_living  \
717  8820903380 2014-07-28  452000.0         6       2.25         2660   
718  8820903380 2015-01-02  730000.0         6       2.25         2660   

     sqft_lot  floors  waterfront  view  ... grade  sqft_above  sqft_basement  \
717     13579     2.0           0     0  ...     7        2660              0   
718     13579     2.0           0     0  ...     7        2660              0   

     yr_built  yr_renovated  zipcode      lat     long  sqft_living15  \
717      1937          1990    98125  47.7142 -122.286           1120   
718      1937          1990    98125  47.7142 -122.286           1120   

     sqft_lot15  
717        8242  
718        8242  

[2 rows x 21 columns]
3598600049 2
              id       date     price  bedrooms  bathrooms  sqft_living  \
9820  3598600049 2014-10-03  124000.0         1       0.75          840   
9821  3598600049 2015-04-24  224000.0         1       0.75          840  

           id       date     price  bedrooms  bathrooms  sqft_living  \
2496  1000102 2014-09-16  280000.0         6        3.0         2400   
2497  1000102 2015-04-22  300000.0         6        3.0         2400   

      sqft_lot  floors  waterfront  view  ... grade  sqft_above  \
2496      9373     2.0           0     0  ...     7        2400   
2497      9373     2.0           0     0  ...     7        2400   

      sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
2496              0      1991             0    98002  47.3262 -122.214   
2497              0      1991             0    98002  47.3262 -122.214   

      sqft_living15  sqft_lot15  
2496           2060        7316  
2497           2060        7316  

[2 rows x 21 columns]
2019200220 2
               id       date     price  bedrooms  bathrooms  sqft_living  \
12955  2019200220 2014-09-23  160000.0         3       2.25         1470   
12956  2019200220 2015-02-26  269000.0         3       2.25         

...

              id       date     price  bedrooms  bathrooms  sqft_living  \
8340  4154300296 2014-09-26  235000.0         3        1.0          960   
8341  4154300296 2015-03-18  545000.0         3        1.0          960   

      sqft_lot  floors  waterfront  view  ... grade  sqft_above  \
8340      5030     1.0           0     0  ...     7         960   
8341      5030     1.0           0     0  ...     7         960   

      sqft_basement  yr_built  yr_renovated  zipcode      lat    long  \
8340              0      1955             0    98118  47.5611 -122.28   
8341              0      1955             0    98118  47.5611 -122.28   

      sqft_living15  sqft_lot15  
8340           1460        5400  
8341           1460        5400  

[2 rows x 21 columns]
726049190 2
            id       date     price  bedrooms  bathrooms  sqft_living  \
823  726049190 2014-10-02  287500.0         3        1.0         1810   
824  726049190 2015-02-18  431000.0         3        1.0         1810

              id       date     price  bedrooms  bathrooms  sqft_living  \
9506  8103000110 2014-06-03  280000.0         2        1.5         1480   
9507  8103000110 2015-02-05  490000.0         2        1.5         1480   

      sqft_lot  floors  waterfront  view  ... grade  sqft_above  \
9506     15641     1.0           0     0  ...     7        1480   
9507     15641     1.0           0     0  ...     7        1480   

      sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
9506              0      1940             0    98146  47.5008 -122.366   
9507              0      1940             0    98146  47.5008 -122.366   

      sqft_living15  sqft_lot15  
9506           1520        7525  
9507           1520        7525  

[2 rows x 21 columns]
3578401060 2
              id       date     price  bedrooms  bathrooms  sqft_living  \
5694  3578401060 2014-12-16  345000.0         3       2.25         1920   
5695  3578401060 2015-05-04  625000.0         3       2.25   

[2 rows x 21 columns]
2568300045 2
              id       date     price  bedrooms  bathrooms  sqft_living  \
9720  2568300045 2014-06-25  305000.0         6        2.0         1900   
9721  2568300045 2015-03-19  649950.0         6        2.0         1900   

      sqft_lot  floors  waterfront  view  ... grade  sqft_above  \
9720      8240     1.0           0     0  ...     7        1200   
9721      8240     1.0           0     0  ...     7        1200   

      sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
9720            700      1964             0    98125  47.7037 -122.296   
9721            700      1964             0    98125  47.7037 -122.296   

      sqft_living15  sqft_lot15  
9720           1900        8240  
9721           1900        8240  

[2 rows x 21 columns]
8648900110 2
               id       date     price  bedrooms  bathrooms  sqft_living  \
20053  8648900110 2014-05-05  555000.0         3        2.5         1940   
20054  8648900110 2014-0

               id       date     price  bedrooms  bathrooms  sqft_living  \
12434  1217000340 2014-06-06  185000.0         3        1.0         1840   
12435  1217000340 2015-02-19  340000.0         3        1.0         1840   

       sqft_lot  floors  waterfront  view  ... grade  sqft_above  \
12434      8100     1.0           0     0  ...     7         920   
12435      8100     1.0           0     0  ...     7         920   

       sqft_basement  yr_built  yr_renovated  zipcode     lat    long  \
12434            920      1953             0    98166  47.455 -122.35   
12435            920      1953             0    98166  47.455 -122.35   

       sqft_living15  sqft_lot15  
12434           1250        8100  
12435           1250        8100  

[2 rows x 21 columns]
2206700215 2
              id       date     price  bedrooms  bathrooms  sqft_living  \
8011  2206700215 2014-08-22  375000.0         4        2.0         2070   
8012  2206700215 2015-04-22  550000.0         4        

[2 rows x 21 columns]
7504021310 2
               id       date     price  bedrooms  bathrooms  sqft_living  \
15001  7504021310 2014-05-06  525000.0         3        2.5         2970   
15002  7504021310 2014-12-04  745000.0         3        2.5         2970   

       sqft_lot  floors  waterfront  view  ... grade  sqft_above  \
15001     11985     1.0           0     0  ...     9        1770   
15002     11985     1.0           0     0  ...     9        1770   

       sqft_basement  yr_built  yr_renovated  zipcode      lat     long  \
15001           1200      1995             0    98074  47.6359 -122.052   
15002           1200      1995             0    98074  47.6359 -122.052   

       sqft_living15  sqft_lot15  
15001           2990       12049  
15002           2990       12049  

[2 rows x 21 columns]
6308000010 2
               id       date     price  bedrooms  bathrooms  sqft_living  \
14982  6308000010 2014-12-08  585000.0         3        2.5         2290   
14983  63080

In [6]:
# Removendo as casas com IDs duplicados, mantendo apenas o registro com a data mais recente

df_houses = dfhouses.sort_values(by=['id', 'date'])
# df_order.head(10)
df_houses.drop_duplicates(subset=['id'], keep='last', inplace=True)

# testando o resultado com um id que eu sei que era duplicado
df_houses.query('id == 795000620')

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
17604,795000620,2015-03-11,157000.0,3,1.0,1080,6250,1.0,0,0,...,5,1080,0,1950,0,98168,47.5045,-122.33,1070,6250


In [7]:
# Estabelecendo nivel para as casas de acordo com o preço

df_houses['nivel'] = df_houses['price'].apply(
lambda preco:
    'nivel_0' if 0 <= preco < 321950 else(
        'nivel_1' if 321950 <= preco < 450000 else(
            'nivel_2' if 450000 <= preco < 645000 else 'nivel_3'
        )
    )
)

In [8]:
df_houses['nivel'].value_counts()

nivel_2    5491
nivel_3    5396
nivel_0    5296
nivel_1    5253
Name: nivel, dtype: int64

In [10]:
!pip install geopy

In [123]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut

In [12]:
# Inicializando Nominatim API

geolocator = Nominatim( user_agent='geoapiExercicises' )

In [24]:
# Testando com uma geolocalização aleatoria

response = geolocator.reverse('47.5045,-122.33')
response.raw

{'place_id': 223286090,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 652545195,
 'lat': '47.5045104',
 'lon': '-122.33015260018033',
 'display_name': '11017, 3rd Avenue South, Beverly Park, White Center, Seattle, King County, Washington, 98168, United States',
 'address': {'house_number': '11017',
  'road': '3rd Avenue South',
  'neighbourhood': 'Beverly Park',
  'suburb': 'White Center',
  'city': 'Seattle',
  'county': 'King County',
  'state': 'Washington',
  'postcode': '98168',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['47.504455', '47.5045659', '-122.3302286', '-122.3300766']}

Adicione as seguintes informações ao imóvel:

 - O nome da Rua
 - O número do imóvel
 - O nome do Bairro
 - O nome da Cidade
 - O nome do Estado

In [21]:
#  Criando novas colunas vazias no dataframe

df_houses['road'] = pd.np.nan
df_houses['house_number'] = pd.np.nan
df_houses['neighbourhood'] = pd.np.nan
df_houses['city'] = pd.np.nan
df_houses['state'] = pd.np.nan

<ipython-input-21-3840b47f0ffe>:3: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df_houses['road'] = pd.np.nan
<ipython-input-21-3840b47f0ffe>:4: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df_houses['house_number'] = pd.np.nan
<ipython-input-21-3840b47f0ffe>:5: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df_houses['neighbourhood'] = pd.np.nan
<ipython-input-21-3840b47f0ffe>:6: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df_houses['city'] = pd.np.nan
<ipython-input-21-3840b47f0ffe>:7: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  df_houses['st

In [93]:
df_houses.tail(20)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,lat,long,sqft_living15,sqft_lot15,nivel,road,house_number,neighbourhood,city,state
19678,9834201375,2015-02-06,425000.0,3,2.25,1420,1230,2.0,0,0,...,47.5703,-122.288,1400,1230,nivel_1,NaN,NaN,NaN,NaN,NaN
6391,9834201470,2014-12-18,303000.0,2,1.50,1000,1075,2.0,0,0,...,47.5708,-122.288,1000,1083,nivel_0,NaN,NaN,NaN,NaN,NaN
11329,9835800320,2014-08-28,300000.0,4,1.75,2080,8750,1.0,0,0,...,47.3749,-122.291,1790,8750,nivel_0,NaN,NaN,NaN,NaN,NaN
4421,9835800750,2014-12-03,247000.0,3,2.25,1640,7630,1.0,0,0,...,47.3739,-122.290,1930,7630,nivel_0,NaN,NaN,NaN,NaN,NaN
5413,9835800840,2015-02-04,215000.0,4,2.00,1470,7000,1.0,0,0,...,47.3742,-122.289,1640,7000,nivel_0,NaN,NaN,NaN,NaN,NaN
11262,9835801000,2014-06-25,245700.0,3,2.25,1640,8400,1.0,0,0,...,47.3733,-122.289,1600,8120,nivel_0,NaN,NaN,NaN,NaN,NaN
10393,9839300125,2015-01-07,575000.0,4,2.00,1810,4400,2.0,0,0,...,47.6132,-122.292,1470,4400,nivel_2,NaN,NaN,NaN,NaN,NaN
16966,9839300285,2015-04-12,720000.0,3,2.50,2100,2200,2.0,0,0,...,47.6140,-122.294,1750,4400,nivel_3,NaN,NaN,NaN,NaN,NaN
7147,9839300545,2014-07-14,605000.0,2,2.00,1270,5500,1.5,0,0,...,47.6121,-122.294,1870,4400,nivel_2,NaN,NaN,NaN,NaN,NaN
14249,9839300775,2014-07-10,655000.0,4,2.25,2170,4080,2.0,0,0,...,47.6124,-122.293,1890,4400,nivel_3,NaN,NaN,NaN,NaN,NaN


In [83]:
# verificando a integridade das colunas lat e long
import numpy as np

print(df_houses['lat'].isnull().sum())
print(df_houses['long'].isnull().sum())

0
0


In [124]:
for i in range( len(df_houses) ):
    
    # Implementando cache caso tenha que rodar de novo o laço for usando a coluna "house_number"
    if pd.isna(df_houses.iloc[i, 23]) == False:
        continue
    
    # Logando o percentual de varredura do dataframe    
    perc = (i/len(df_houses))*100
    if perc >= 1 and round(perc,2) % 2 == 0:
        print(f'{perc:.3f}%')
    
    # Montando a string de busca com os dados de 'lat' e 'long'
    query = str(df_houses.iloc[i, 17]) + ',' + str(df_houses.iloc[i, 18])
    
    # API request
    try:
        response = geolocator.reverse(query)
    except GeocoderTimedOut:
        continue
    
    # Popula o dataframe com os dados da API
    if 'road' in response.raw['address']:
        df_houses.iloc[i, 22] = response.raw['address']['road']
    
    if 'house_number' in response.raw['address']:
        df_houses.iloc[i, 23] = response.raw['address']['house_number']
        
    if 'neighbourhood' in response.raw['address']:
        df_houses.iloc[i, 24] = response.raw['address']['neighbourhood']
        
    if 'city' in response.raw['address']:
        df_houses.iloc[i, 25] = response.raw['address']['city']
        
    if 'state' in response.raw['address']:
        df_houses.iloc[i, 26] = response.raw['address']['state']
        

9.997%
10.002%
18.002%
25.998%
26.003%
28.000%
28.004%
29.996%
30.001%
31.998%
32.002%
33.999%
34.004%
35.996%
36.000%
36.005%
37.997%
38.001%
39.998%
40.003%
41.999%
42.004%
43.996%
44.001%
45.997%
46.002%
47.999%
48.003%
49.995%
50.000%
50.005%
51.997%
52.001%
53.998%
54.003%
55.999%
56.004%
57.996%
58.001%
59.997%
60.002%
61.999%
62.003%
63.995%
64.000%
64.004%
65.996%
66.001%
67.998%
68.002%
69.999%
70.004%
71.996%
72.000%
73.997%
74.002%
75.998%
76.003%
78.000%
78.004%
79.996%
80.001%
81.998%
82.002%
83.999%
84.004%
85.996%
86.000%
86.005%
87.997%
88.001%
89.998%
90.003%
91.999%
92.004%
93.996%
94.001%
95.997%
96.002%
98.003%
99.995%


In [126]:
df_houses.tail()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,lat,long,sqft_living15,sqft_lot15,nivel,road,house_number,neighbourhood,city,state
16737,9842300095,2014-07-25,365000.0,5,2.00,1600,4168,1.5,0,0,...,47.5297,-122.381,1190,4168,nivel_1,Southwest Southern Street,3739,High Point,Seattle,Washington
3260,9842300485,2015-03-11,380000.0,2,1.00,1040,7372,1.0,0,0,...,47.5285,-122.378,1930,5150,nivel_1,Southwest Thistle Street,3526,NaN,Seattle,Washington
7621,9842300540,2014-06-24,339000.0,3,1.00,1100,4128,1.0,0,0,...,47.5296,-122.379,1510,4538,nivel_1,37th Avenue Southwest,8202,High Point,Seattle,Washington
20979,9895000040,2014-07-03,399900.0,2,1.75,1410,1005,1.5,0,0,...,47.5446,-122.018,1440,1188,nivel_1,Northeast High Street,930,NaN,NaN,Washington
15951,9900000190,2014-10-30,268950.0,3,1.00,1320,8100,1.0,0,0,...,47.4697,-122.351,1000,8100,nivel_0,Southwest 149th Street,1220,Seahurst,NaN,Washington


In [127]:
# salvando num arquivo pra não precisar fazer de novo
df_houses.to_csv('dados/processados/kc_house_data_comp.csv')